# Mesh generation for tutorial 06, case 1b

This file generates the mesh which is used in the following examples:
* 1b_poisson

The test case is from section 5.1 of
```
F. Negri, G. Rozza, A. Manzoni and A. Quarteroni. Reduced Basis Method for Parametrized Elliptic Optimal Control Problems. SIAM Journal on Scientific Computing, 35(5): A2316-A2340, 2013.
```

In [ ]:
import numpy as np
import pygmsh
from mpi4py import MPI
from dolfinx import cpp
from dolfinx.mesh import create_mesh, create_meshtags
from dolfinx.io import ufl_mesh_from_gmsh, XDMFFile
from dolfinx.plotting import plot

In [ ]:
assert MPI.COMM_WORLD.size == 1, "This mesh generation notebook is supposed to be run in serial"

### Geometrical parameters and related quantities

In [ ]:
L1 = 1.0
L2 = 3.0
H = 1.0
lcar = 0.05

### Create mesh

In [ ]:
geom = pygmsh.geo.Geometry()
p0 = geom.add_point([0.0, 0.0, 0.0], lcar)
p1 = geom.add_point([L1, 0.0, 0.0], lcar)
p2 = geom.add_point([L1 + L2, 0.0, 0.0], lcar)
p3 = geom.add_point([L1 + L2, H, 0.0], lcar)
p4 = geom.add_point([L1, H, 0.0], lcar)
p5 = geom.add_point([0.0, H, 0.0], lcar)
l0 = geom.add_line(p0, p1)
l1 = geom.add_line(p1, p4)
l2 = geom.add_line(p4, p5)
l3 = geom.add_line(p5, p0)
l4 = geom.add_line(p1, p2)
l5 = geom.add_line(p2, p3)
l6 = geom.add_line(p3, p4)
geom.add_physical([l0, l4, l5, l6, l2, l3], label=1)
line_loop_rectangle_left = geom.add_line_loop([l0, l1, l2, l3])
line_loop_rectangle_right = geom.add_line_loop([l4, l5, l6, -l1])
rectangle_left = geom.add_plane_surface(line_loop_rectangle_left)
rectangle_right = geom.add_plane_surface(line_loop_rectangle_right)
geom.add_physical(rectangle_left, label=11)
geom.add_physical(rectangle_right, label=12)
pygmsh_mesh = pygmsh.generate_mesh(geom)

### Convert to a dolfinx mesh

In [ ]:
cells, x = pygmsh_mesh.cells_dict["triangle"], pygmsh_mesh.points[:, :2]
mesh = create_mesh(MPI.COMM_WORLD, cells, x, ufl_mesh_from_gmsh("triangle", x.shape[1]))

In [ ]:
plot(mesh)

### Extract subdomains as MeshTags

In [ ]:
subdomains_entities = cells
subdomains_values = pygmsh_mesh.cell_data_dict["gmsh:physical"]["triangle"] - 10
subdomains = create_meshtags(mesh, mesh.topology.dim,
                             cpp.graph.AdjacencyList_int32(subdomains_entities), np.int32(subdomains_values))
subdomains.name = "subdomains"

### Extract boundaries as MeshTags

In [ ]:
boundaries_entities = pygmsh_mesh.cells_dict["line"]
boundaries_values = pygmsh_mesh.cell_data_dict["gmsh:physical"]["line"]
mesh.topology.create_connectivity(mesh.topology.dim - 1, 0)
boundaries = create_meshtags(mesh, mesh.topology.dim - 1,
                             cpp.graph.AdjacencyList_int32(boundaries_entities), np.int32(boundaries_values))
boundaries.name = "boundaries"

### Save mesh, subdomains and boundaries

In [ ]:
with XDMFFile(MPI.COMM_WORLD, "rectangle.xdmf", "w") as file:
    file.write_mesh(mesh)
    file.write_meshtags(subdomains)
    mesh.topology.create_connectivity(mesh.topology.dim - 1, mesh.topology.dim)
    file.write_meshtags(boundaries)